In [1]:
OUTPUT_BUCKET_FOLDER = "gs://kaggle-ocp-data/output/"
DATA_BUCKET_FOLDER = "gs://kaggle-ocp-data/data/"

In [2]:
from pyspark.sql.types import *
import pyspark.sql.functions as F

# Loading data

In [3]:
truncate_day_from_timestamp_udf = F.udf(lambda ts: int(ts / 1000 / 60 / 60 / 24), IntegerType())

In [4]:
events_schema = StructType(
                    [StructField("display_id", IntegerType(), True),
                    StructField("uuid_event", StringType(), True),                    
                    StructField("document_id_event", IntegerType(), True),
                    StructField("timestamp_event", IntegerType(), True),
                    StructField("platform_event", IntegerType(), True),
                    StructField("geo_location_event", StringType(), True)]
                    )

events_df = spark.read.schema(events_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER + "events.csv") \
                .withColumn('day_event', truncate_day_from_timestamp_udf('timestamp_event')) \
                .alias('events')

In [38]:
events_df.show() #day_event는 timestmap를 truncate_day_from_timestamp_udf를 사용하여, 86400000으로 나눠, 날짜(day) 추출

+----------+--------------+-----------------+---------------+--------------+------------------+---------+
|display_id|    uuid_event|document_id_event|timestamp_event|platform_event|geo_location_event|day_event|
+----------+--------------+-----------------+---------------+--------------+------------------+---------+
|         1|cb8c55702adb93|           379743|             61|             3|         US>SC>519|        0|
|         2|79a85fa78311b9|          1794259|             81|             2|         US>CA>807|        0|
|         3|822932ce3d8757|          1179111|            182|             2|         US>MI>505|        0|
|         4|85281d0a49f7ac|          1777797|            234|             2|         US>WV>564|        0|
|         5|8d0daef4bf5b56|           252458|            338|             2|             SG>00|        0|
|         6|7765b4faae4ad4|          1773517|            395|             3|         US>OH>510|        0|
|         7|2cc3f6457d16da|          1149661| 

In [40]:
print((events_df.count(), len(events_df.columns)))

(23120126, 7)


In [24]:
events_df.groupBy('day_event').count().show()

+---------+-------+
|day_event|  count|
+---------+-------+
|       12|1477939|
|        1|1516879|
|       13|1672275|
|        6|1699343|
|        3|1486649|
|        5|1319657|
|        9|1614693|
|        4|1310388|
|        8|1616280|
|        7|1596045|
|       10|1583387|
|       11|1356708|
|       14|1595929|
|        2|1573079|
|        0|1700875|
+---------+-------+



In [8]:
promoted_content_schema = StructType(
                    [StructField("ad_id", IntegerType(), True),
                    StructField("document_id_promo", IntegerType(), True),                    
                    StructField("campaign_id", IntegerType(), True),
                    StructField("advertiser_id", IntegerType(), True)]
                    )

promoted_content_df = spark.read.schema(promoted_content_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER+"promoted_content.csv") \
                .alias('promoted_content')

In [41]:
print((promoted_content_df.count(), len(promoted_content_df.columns)))

(559583, 4)


In [42]:
promoted_content_df.show()

+-----+-----------------+-----------+-------------+
|ad_id|document_id_promo|campaign_id|advertiser_id|
+-----+-----------------+-----------+-------------+
|    1|             6614|          1|            7|
|    2|           471467|          2|            7|
|    3|             7692|          3|            7|
|    4|           471471|          2|            7|
|    5|           471472|          2|            7|
|    6|            12736|          1|            7|
|    7|            12808|          1|            7|
|    8|           471477|          2|            7|
|    9|            13379|          1|            7|
|   10|            13885|          1|            7|
|   11|            14230|          1|            7|
|   12|           446701|         10|           19|
|   13|           471499|         10|           19|
|   14|           471500|         10|           19|
|   15|           471501|         10|           19|
|   16|           471514|         17|           19|
|   17|     

In [25]:
clicks_train_schema = StructType(
                    [StructField("display_id", IntegerType(), True),
                    StructField("ad_id", IntegerType(), True),                    
                    StructField("clicked", IntegerType(), True)]
                    )

clicks_train_df = spark.read.schema(clicks_train_schema).options(header='true', inferschema='false', nullValue='\\N') \
                .csv(DATA_BUCKET_FOLDER+"clicks_train.csv") \
                .alias('clicks_train')

In [43]:
print((clicks_train_df.count(), len(clicks_train_df.columns)))

(87141731, 3)


In [44]:
clicks_train_df.show()

+----------+------+-------+
|display_id| ad_id|clicked|
+----------+------+-------+
|         1| 42337|      0|
|         1|139684|      0|
|         1|144739|      1|
|         1|156824|      0|
|         1|279295|      0|
|         1|296965|      0|
|         2|125211|      0|
|         2|156535|      0|
|         2|169564|      0|
|         2|308455|      1|
|         3| 71547|      0|
|         3| 95814|      0|
|         3|152141|      0|
|         3|183846|      0|
|         3|228657|      1|
|         3|250082|      0|
|         4|149930|      0|
|         4|153623|      1|
|         4|184709|      0|
|         4|186849|      0|
+----------+------+-------+
only showing top 20 rows



In [27]:
clicks_train_joined_df = clicks_train_df \
                         .join(promoted_content_df, on='ad_id', how='left') \
                         .join(events_df, on='display_id', how='left')                         
clicks_train_joined_df.createOrReplaceTempView('clicks_train_joined')

In [37]:
print((clicks_train_joined_df.count(), len(clicks_train_joined_df.columns)))

(87141731, 12)


In [35]:
clicks_train_joined_df.show()

+----------+------+-------+-----------------+-----------+-------------+--------------+-----------------+---------------+--------------+------------------+---------+
|display_id| ad_id|clicked|document_id_promo|campaign_id|advertiser_id|    uuid_event|document_id_event|timestamp_event|platform_event|geo_location_event|day_event|
+----------+------+-------+-----------------+-----------+-------------+--------------+-----------------+---------------+--------------+------------------+---------+
|       148|152656|      0|          1086755|      10511|         2151|9adce6a5363308|          1205772|          11202|             2|         US>LA>612|        0|
|       148|152140|      0|          1060089|      19032|         1593|9adce6a5363308|          1205772|          11202|             2|         US>LA>612|        0|
|       148| 89351|      1|           990613|       7617|         2181|9adce6a5363308|          1205772|          11202|             2|         US>LA>612|        0|
|       14

In [59]:
validation_display_ids_df = clicks_train_joined_df.select('display_id','day_event').distinct() \
                                .sampleBy("day_event", fractions={0: 0.2, 1: 0.2, 2: 0.2, 3: 0.2, 4: 0.2, \
                                                                5: 0.2, 6: 0.2, 7: 0.2, 8: 0.2, 9: 0.2, 10: 0.2, \
                                                               11: 1.0, 12: 1.0}, seed=0)   


In [60]:
print((validation_display_ids_df.count(), len(validation_display_ids_df.columns)))

(5303867, 2)


In [61]:
validation_display_ids_df.show()

+----------+---------+
|display_id|day_event|
+----------+---------+
|      2122|        0|
|      2659|        0|
|      3794|        0|
|      3918|        0|
|      5156|        0|
|      5518|        0|
|      6357|        0|
|      6654|        0|
|      7240|        0|
|      7833|        0|
|      8086|        0|
|      9427|        0|
|      9900|        0|
|     10206|        0|
|     11317|        0|
|     12799|        0|
|     13623|        0|
|     15846|        0|
|     16574|        0|
|     17389|        0|
+----------+---------+
only showing top 20 rows



In [62]:
validation_display_ids_df.groupBy('day_event').count().show()

+---------+-------+
|day_event|  count|
+---------+-------+
|       12|1256811|
|        1| 257681|
|        6| 288947|
|        3| 253228|
|        5| 224663|
|        9| 275181|
|        4| 222963|
|        8| 275006|
|        7| 271356|
|       10| 268112|
|       11|1153206|
|        2| 267407|
|        0| 289306|
+---------+-------+



In [66]:
validation_display_ids_df.createOrReplaceTempView("validation_display_ids") 
# createOrReplaceTempView는 현재 메모리가 존재하지 않는 테이블의 임시 뷰를 생성했지만, 그 위에 SQL 쿼리를 실행 할 수 있음

In [67]:
validation_set_df = spark.sql('''SELECT display_id, ad_id, uuid_event, day_event, timestamp_event,
                                        document_id_promo, platform_event, geo_location_event FROM clicks_train_joined t
             WHERE EXISTS (SELECT display_id FROM validation_display_ids 
                           WHERE display_id = t.display_id)''')

# validation_display_ids_df에 랜덤 추출한 display_id 기준으로 display_id가 일치하는 
# clicks_train_joined의 display_id, ad_id, uuid_event, day_event, timestamp_event,document_id_promo, platform_event, geo_location_event를 join 시킴

In [68]:
validation_set_df.show()

+----------+------+--------------+---------+---------------+-----------------+--------------+------------------+
|display_id| ad_id|    uuid_event|day_event|timestamp_event|document_id_promo|platform_event|geo_location_event|
+----------+------+--------------+---------+---------------+-----------------+--------------+------------------+
|      2122| 36619|7ceed8e24a87d7|        0|         148795|           899906|             3|             SG>00|
|      2122| 81643|7ceed8e24a87d7|        0|         148795|          1094108|             3|             SG>00|
|      2122|216100|7ceed8e24a87d7|        0|         148795|          1548042|             3|             SG>00|
|      2659| 55819|964e40766c3f39|        0|         185389|           986576|             3|             CA>BC|
|      2659| 76816|964e40766c3f39|        0|         185389|           824972|             3|             CA>BC|
|      2659| 90013|964e40766c3f39|        0|         185389|          1117722|             3|   

In [69]:
print((validation_set_df.count(), len(validation_set_df.columns)))

(27380257, 8)


In [70]:
# GCS 버킷에 validation_set 파케이 형식으로 저장
validation_set_gcs_output = "validation_set.parquet"
validation_set_df.write.parquet(OUTPUT_BUCKET_FOLDER+validation_set_gcs_output, mode='overwrite')

* 파케이(Parquet)는 하둡에서 컬럼방식으로 저장 포맷(데이터프레임이 저장되면, 스키마 정보를 유지한 채로 데이터를 저장)
* 파케이는 프로그래밍 언어, 데이터 모델, 혹은 데이터 처리 엔진과 독립적으로 엔진과 하둡 생태계에 속한 프로젝트에서 컬럼방식으로 데이터를 효율적으로 저장하여 처리성능을 비약적으로 향상시킴